In [13]:
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
import tensorflow as tf
import tensorflow_federated as tff
import nest_asyncio
nest_asyncio.apply()


In [2]:
def perform_differential_testing(predictions_i, predictions_j, labels, data_type):
    if predictions_i.ndim == 1:
        predictions_i = np.expand_dims(predictions_i, axis=1)
    if predictions_j.ndim == 1:
        predictions_j = np.expand_dims(predictions_j, axis=1)
    
    pred_class_i = np.argmax(predictions_i, axis=1)
    pred_class_j = np.argmax(predictions_j, axis=1)
    
    Δ_class = np.sum(pred_class_i != pred_class_j)
    Δ_score = np.sum(predictions_i != predictions_j)
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
    contingency = np.array([[np.sum((pred_class_i == k) & (pred_class_j == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    return Δ_class, Δ_score, P_KS, P_X2

In [3]:
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])


In [4]:
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=(tf.TensorSpec(shape=[None, 28, 28], dtype=tf.float32),
                    tf.TensorSpec(shape=[None], dtype=tf.int32)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


In [5]:
def create_fake_model(bug=False):
    model = create_model()
    if bug:
        optimizer = tf.keras.optimizers.Adam(learning_rate=10.0)  # Extremely high learning rate
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Normal learning rate
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [6]:
def is_outlier(metric_data, epsilon=0.1, min_samples=3):
    num_points = metric_data.shape[0]
    distances = np.linalg.norm(metric_data[:, np.newaxis] - metric_data, axis=2)
    neighbors = np.sum(distances < epsilon, axis=1)
    outliers = neighbors < min_samples
    return outliers


In [7]:
# Simulate predictions for 10 clients
num_clients = 10
predictions = [np.random.rand(100, 10) for _ in range(num_clients)]  # Random predictions for simplicity

# Calculate Δ_class for each pair of clients
Δ_class_matrix = np.zeros((num_clients, num_clients))
for i in range(num_clients):
    for j in range(i + 1, num_clients):
        Δ_class, _, _, _ = perform_differential_testing(predictions[i], predictions[j], None, None)
        Δ_class_matrix[i, j] = Δ_class
        Δ_class_matrix[j, i] = Δ_class

# Print the matrix
print("Δ_class Matrix:")
print(Δ_class_matrix)


Δ_class Matrix:
[[ 0. 92. 87. 95. 91. 90. 89. 93. 92. 89.]
 [92.  0. 86. 92. 92. 90. 90. 87. 91. 92.]
 [87. 86.  0. 88. 92. 92. 91. 86. 90. 85.]
 [95. 92. 88.  0. 81. 91. 87. 87. 93. 92.]
 [91. 92. 92. 81.  0. 92. 89. 94. 93. 94.]
 [90. 90. 92. 91. 92.  0. 90. 92. 90. 88.]
 [89. 90. 91. 87. 89. 90.  0. 87. 93. 89.]
 [93. 87. 86. 87. 94. 92. 87.  0. 88. 91.]
 [92. 91. 90. 93. 93. 90. 93. 88.  0. 90.]
 [89. 92. 85. 92. 94. 88. 89. 91. 90.  0.]]


In [10]:
def client_optimizer_fn():
    return tf.keras.optimizers.Adam(learning_rate=0.001)


In [14]:
# Define the federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Simulate federated training
num_rounds = 10  # Define the number of rounds
num_clients = 10  # Define the number of clients

for round_num in range(1, num_rounds + 1):
    # Generate client data (for simplicity, using random data here)
    client_data = [np.random.rand(100, 28, 28) for _ in range(num_clients)]
    client_labels = [np.random.randint(0, 10, 100) for _ in range(num_clients)]
    
    # Create a list of client datasets
    federated_data = [
        tff.simulation.ClientData.from_clients_and_fn(
            client_ids=[str(i)],
            create_tf_dataset_for_client_fn=lambda _: tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i])).batch(20)
        ) for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Perform differential testing
    predictions = [model.predict(client_data[i]) for i in range(num_clients)]
    Δ_class_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            Δ_class, _, _, _ = perform_differential_testing(predictions[i], predictions[j], None, None)
            Δ_class_matrix[i, j] = Δ_class
            Δ_class_matrix[j, i] = Δ_class
    
    # Detect outliers
    outliers = is_outlier(Δ_class_matrix)
    print(f'Round {round_num}, Outliers: {outliers}')


TypeError: Expected tensorflow.python.data.ops.dataset_ops.DatasetV2 or tensorflow.python.data.ops.dataset_ops.DatasetV1, found tensorflow_federated.python.simulation.client_data.ConcreteClientData.

In [9]:
# Simulate federated training with differential testing and outlier detection
for round_num in range(1, num_rounds + 1):
    # Generate client data (for simplicity, using random data here)
    client_data = [np.random.rand(100, 28, 28) for _ in range(num_clients)]
    client_labels = [np.random.randint(0, 10, 100) for _ in range(num_clients)]
    
    # Create a list of client datasets
    federated_data = [
        tff.simulation.ClientData.from_clients_and_fn(
            client_ids=[str(i)],
            create_tf_dataset_for_client_fn=lambda _: tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i])).batch(20)
        ) for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Perform differential testing
    predictions = [model.predict(client_data[i]) for i in range(num_clients)]
    Δ_class_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            Δ_class, _, _, _ = perform_differential_testing(predictions[i], predictions[j], None, None)
            Δ_class_matrix[i, j] = Δ_class
            Δ_class_matrix[j, i] = Δ_class
    
    # Detect outliers
    outliers = is_outlier(Δ_class_matrix)
    print(f'Round {round_num}, Outliers: {outliers}')


NameError: name 'num_rounds' is not defined

In [15]:
import tensorflow_federated as tff
import tensorflow as tf
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
import nest_asyncio

nest_asyncio.apply()

# Set the local execution context
tff.backends.native.set_local_execution_context()

# Define the metrics function
def perform_differential_testing(predictions_i, predictions_j, labels, data_type):
    if predictions_i.ndim == 1:
        predictions_i = np.expand_dims(predictions_i, axis=1)
    if predictions_j.ndim == 1:
        predictions_j = np.expand_dims(predictions_j, axis=1)
    
    pred_class_i = np.argmax(predictions_i, axis=1)
    pred_class_j = np.argmax(predictions_j, axis=1)
    
    Δ_class = np.sum(pred_class_i != pred_class_j)
    Δ_score = np.sum(predictions_i != predictions_j)
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
    contingency = np.array([[np.sum((pred_class_i == k) & (pred_class_j == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    return Δ_class, Δ_score, P_KS, P_X2

# Create a simple model
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Create a federated learning process
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=(tf.TensorSpec(shape=[None, 28, 28], dtype=tf.float32),
                    tf.TensorSpec(shape=[None], dtype=tf.int32)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Define the client optimizer function
def client_optimizer_fn():
    return tf.keras.optimizers.Adam(learning_rate=0.001)

# Define the federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Simulate federated training
num_rounds = 10  # Define the number of rounds
num_clients = 10  # Define the number of clients

for round_num in range(1, num_rounds + 1):
    # Generate client data (for simplicity, using random data here)
    client_data = [np.random.rand(100, 28, 28) for _ in range(num_clients)]
    client_labels = [np.random.randint(0, 10, 100) for _ in range(num_clients)]
    
    # Create a list of client datasets
    federated_data = [
        tff.simulation.ClientData.from_clients_and_fn(
            client_ids=[str(i)],
            create_tf_dataset_for_client_fn=lambda _: tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i])).batch(20)
        ) for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Perform differential testing
    predictions = [model.predict(client_data[i]) for i in range(num_clients)]
    Δ_class_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            Δ_class, _, _, _ = perform_differential_testing(predictions[i], predictions[j], None, None)
            Δ_class_matrix[i, j] = Δ_class
            Δ_class_matrix[j, i] = Δ_class
    
    # Detect outliers
    outliers = is_outlier(Δ_class_matrix)
    print(f'Round {round_num}, Outliers: {outliers}')


TypeError: Expected tensorflow.python.data.ops.dataset_ops.DatasetV2 or tensorflow.python.data.ops.dataset_ops.DatasetV1, found tensorflow_federated.python.simulation.client_data.ConcreteClientData.

In [16]:
import tensorflow_federated as tff
import tensorflow as tf
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
import nest_asyncio

nest_asyncio.apply()

# Set the local execution context
tff.backends.native.set_local_execution_context()

# Define the metrics function
def perform_differential_testing(predictions_i, predictions_j, labels, data_type):
    if predictions_i.ndim == 1:
        predictions_i = np.expand_dims(predictions_i, axis=1)
    if predictions_j.ndim == 1:
        predictions_j = np.expand_dims(predictions_j, axis=1)
    
    pred_class_i = np.argmax(predictions_i, axis=1)
    pred_class_j = np.argmax(predictions_j, axis=1)
    
    Δ_class = np.sum(pred_class_i != pred_class_j)
    Δ_score = np.sum(predictions_i != predictions_j)
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
    contingency = np.array([[np.sum((pred_class_i == k) & (pred_class_j == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    return Δ_class, Δ_score, P_KS, P_X2

# Create a simple model
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Create a federated learning process
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=(tf.TensorSpec(shape=[None, 28, 28], dtype=tf.float32),
                    tf.TensorSpec(shape=[None], dtype=tf.int32)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Define the client optimizer function
def client_optimizer_fn():
    return tf.keras.optimizers.Adam(learning_rate=0.001)

# Define the federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Simulate federated training
num_rounds = 10  # Define the number of rounds
num_clients = 10  # Define the number of clients

for round_num in range(1, num_rounds + 1):
    # Generate client data (for simplicity, using random data here)
    client_data = [np.random.rand(100, 28, 28) for _ in range(num_clients)]
    client_labels = [np.random.randint(0, 10, 100) for _ in range(num_clients)]
    
    # Create a list of client datasets
    federated_data = [
        tff.simulation.ClientData.from_clients_and_fn(
            client_ids=[str(i)],
            create_tf_dataset_for_client_fn=lambda _: tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i])).batch(20)
        ) for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Perform differential testing
    predictions = [model.predict(client_data[i]) for i in range(num_clients)]
    Δ_class_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            Δ_class, _, _, _ = perform_differential_testing(predictions[i], predictions[j], None, None)
            Δ_class_matrix[i, j] = Δ_class
            Δ_class_matrix[j, i] = Δ_class
    
    # Detect outliers
    outliers = is_outlier(Δ_class_matrix)
    print(f'Round {round_num}, Outliers: {outliers}')


TypeError: Expected tensorflow.python.data.ops.dataset_ops.DatasetV2 or tensorflow.python.data.ops.dataset_ops.DatasetV1, found tensorflow_federated.python.simulation.client_data.ConcreteClientData.

In [17]:
import tensorflow_federated as tff
import tensorflow as tf
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
import nest_asyncio

nest_asyncio.apply()

# Set the local execution context
tff.backends.native.set_local_execution_context()

# Define the metrics function
def perform_differential_testing(predictions_i, predictions_j, labels, data_type):
    if predictions_i.ndim == 1:
        predictions_i = np.expand_dims(predictions_i, axis=1)
    if predictions_j.ndim == 1:
        predictions_j = np.expand_dims(predictions_j, axis=1)
    
    pred_class_i = np.argmax(predictions_i, axis=1)
    pred_class_j = np.argmax(predictions_j, axis=1)
    
    Δ_class = np.sum(pred_class_i != pred_class_j)
    Δ_score = np.sum(predictions_i != predictions_j)
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
    contingency = np.array([[np.sum((pred_class_i == k) & (pred_class_j == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    return Δ_class, Δ_score, P_KS, P_X2

# Create a simple model
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Create a federated learning process
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=(tf.TensorSpec(shape=[None, 28, 28], dtype=tf.float32),
                    tf.TensorSpec(shape=[None], dtype=tf.int32)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Define the client optimizer function
def client_optimizer_fn():
    return tf.keras.optimizers.Adam(learning_rate=0.001)

# Define the federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Simulate federated training
num_rounds = 10  # Define the number of rounds
num_clients = 10  # Define the number of clients

for round_num in range(1, num_rounds + 1):
    # Generate client data (using random data here for simplicity)
    client_data = [np.random.rand(100, 28, 28) for _ in range(num_clients)]
    client_labels = [np.random.randint(0, 10, 100) for _ in range(num_clients)]
    
    # Create TensorFlow datasets for each client
    federated_data = [
        tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i])).batch(20)
        for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Create a model to predict on the client data
    model = create_model()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # Get predictions for each client
    predictions = [model.predict(client_data[i]) for i in range(num_clients)]
    
    # Initialize the Δ_class matrix
    Δ_class_matrix = np.zeros((num_clients, num_clients))
    
    # Calculate the Δ_class for each pair of clients
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            Δ_class, _, _, _ = perform_differential_testing(predictions[i], predictions[j], None, None)
            Δ_class_matrix[i, j] = Δ_class
            Δ_class_matrix[j, i] = Δ_class
    
    # Print the Δ_class matrix
    print(f"Round {round_num} Δ_class matrix:")
    print(Δ_class_matrix)
    
    # Detect outliers using the custom DBSCAN-like function
    outliers = is_outlier(Δ_class_matrix)
    print(f'Round {round_num}, Outliers: {outliers}')


c:\Anaconda\envs\tf_fed_env\lib\site-packages\tensorflow\python\framework\c_api_util.py:46: RuntimeWarning: coroutine 'wrap_coroutine_in_current_trace_context.<locals>._wrapped' was never awaited
  self.graph = c_api.TF_NewGraph()
c:\Anaconda\envs\tf_fed_env\lib\site-packages\tensorflow\python\framework\c_api_util.py:46: RuntimeWarning: coroutine '_invoke' was never awaited
  self.graph = c_api.TF_NewGraph()


TypeNotAssignableError: Type `<float64[?,28,28],int32[?]>*` is not assignable to type `<float32[?,28,28],int32[?]>*`